# Tutorial 02: Cell-Centered Finite Volume Method

This tutorial solves the same partial differential equation (PDE) as tutorial 01, namely
a nonlinear Poisson equation, with the following differences:

1. Implements a cell-centered finite volume method with two-point flux approximation as an example of a non-conforming scheme.
2. Implements *all* possible methods of a local operator.


# PDE Problem

Consider the following nonlinear Poisson equation (the same as in tutorial 01) with Dirichlet and Neumann boundary conditions:
\begin{equation} \label{eq:ProblemStrong}
\begin{aligned}
-\Delta u + q(u) &= f &&\text{in $\Omega$},\\
u &= g &&\text{on $\Gamma_D\subseteq\partial\Omega$},\\
-\nabla u\cdot \nu &= j &&\text{on $\Gamma_N=\partial\Omega\setminus\Gamma_D$}.
\end{aligned}
\end{equation}
$\Omega\subset\mathbb{R}^d$ is a domain, $q:\mathbb{R}\to\mathbb{R}$ is a given, possibly
nonlinear function and $f: \Omega\to\mathbb{R}$ is the source term and
$\nu$ denotes the unit outer normal to the domain.

# Cell-centered Finite Volume Method

The application of the cell-centered finite volume method as presented here is
restricted to *axiparallel meshes*. We assume that  the domain $\Omega$ is covered by a mesh
$\mathcal{T}_h = \{T_1, \ldots, T_M\}$ consisting of elements
which are closed sets satisfying
\begin{equation}
\bigcup_{T\in \mathcal{T}_h} T = \overline{\Omega}, \quad
\forall T, T' \in \mathcal{T}_h, T\neq T' : \mathring{T} \cap \mathring{T}' = \emptyset .
\end{equation}
In order to describe the method some further notation is needed.
The nonempty intersections $F = T_F^-\cap T_F^+$
of codimension 1 form the interior skeleton $\mathcal{F}_h^i=\{F_1,\ldots,F_N\}$.
Each intersection is equipped with a unit normal vector $\nu_F$ pointing from $T_F^-$ to $T_F^+$.
The intersections of an element $F=T_F^-\cap\partial\Omega$ with the domain
boundary form the set of boundary intersections $\mathcal{F}_h^{\partial\Omega}=
\{F_1,\ldots,F_L\}$ which can be further partitioned into
Dirichlet boundary intersections $\mathcal{F}_h^{\Gamma_D}$
and Neumann boundary intersections $\mathcal{F}_h^{\Gamma_N}$.
Each boundary intersection is equipped with a unit normal vector
$\nu_F$ which coincides with the unit outer normal to the domain.
Furthermore, $x_T$, $x_F$ denotes the center point of an element or face.
This notation is illustrated graphically in Figure \ref{fig:MeshNotation}.

For the cell-centered finite volume method the discrete function space involved
is the space of piecewise constant functions on the mesh:
\begin{equation*}
W_h = \{w\in L^2(\Omega) \,:\,  \text{$w|_T=$ const for all $T\in\mathcal{T}_h$}\} .
\end{equation*}

In order to derive the residual form we proceed as follows: multiply
equation \eqref{eq:ProblemStrong} with a test function $v\in W_h$, i.e. \textit{from
the discrete space}, and use integration by parts:
\begin{align*}
\int_{\Omega} f v \,dx &= \int_{\Omega} [-\Delta u + q(u)] v\,dx\\
&= \sum_{T\in\mathcal{T}_h} v \int_T -\Delta u + q(u) \,dx &&\text{($v$ const on $T$)}\\
&= \sum_{T\in\mathcal{T}_h} \left[\int_T q(u) v \,dx - \int_{\partial T} \nabla u \cdot \nu v \,ds
\right] &&\text{(Gauss' thm.)} \\
&= \sum_{T\in\mathcal{T}_h} \int_T q(u) v \,dx
-\sum_{F\in\mathcal{F}_h^i} \int_F \nabla u \cdot \nu_F \bigl[v(x_{T_F^-}) - v(x_{T_F^+})\bigr] \,ds \\
& \hspace{10mm}-\sum_{F\in\mathcal{F}_h^{\partial\Omega}} \int_F \nabla u \cdot \nu_F \,ds .
&&\text{(rearrange)}
\end{align*}
At this point, the normal derivative $\partial_{\nu_F} u = \nabla u\cdot \nu_F$
is approximated by a difference quotient
\begin{equation*}
\nabla u\cdot \nu_F = \frac{u_h(x_{T_F^+})-u_h(x_{T_F^-})}{\|x_{T_F^+} - x_{T_F^-}\|}
 + \text{ error}
\end{equation*}
and all integrals are approximated by the midpoint rule
\begin{equation*}
\int_T f \,dx = f(x_T)|T| + \text{ error}
\end{equation*}
where $|T|$ is the measure of $T$.

Put together the cell-centered finite volume method can be stated
in its abstract form suitable for implementation in PDELab:
\begin{equation}
\boxed{ \text{Find $u_h\in W_h$ s.t.:} \quad r_h^{\text{CCFV}}(u_h,v) = 0 \quad \forall v \in W_h }
\end{equation}
where the residual form is
\begin{equation}
\label{eq:res_form_final}
\begin{split}
r_h^{\text{CCFV}}(u_h,v)
& = \sum_{T\in\mathcal{T}_h} q(u_h(x_T)) v(x_T) |T|
- \sum_{T\in\mathcal{T}_h} f(x_T) v(x_T) |T|\\
&\ - \sum_{F\in\mathcal{F}_h^i}
\frac{u_h(x_{T_F^+})-u_h(x_{T_F^-})}{\|x_{T_F^+} - x_{T_F^-}\|}
\bigl[v(x_{T_F^-}) - v(x_{T_F^+})\bigr] |F|\\
&\ + \sum_{F\in\mathcal{F}_h^{\partial\Omega}\cap\Gamma_D}
\frac{u_h(x_{T_F^-})}{\|x_{F} - x_{T_F^-}\|} v(x_{T_F^-}) |F| \\
&\ - \sum_{F\in\mathcal{F}_h^{\partial\Omega}\cap\Gamma_D}
\frac{g(x_{F})}{\|x_{F} - x_{T_F^-}\|} v(x_{T_F^-}) |F|
+ \sum_{F\in\mathcal{F}_h^{\partial\Omega}\cap\Gamma_N} j(x_{F}) v(x_{T_F^-}) |F| .
\end{split}
\end{equation}
In this case *five* different types of integrals are involved in the
residual form:

1. Volume integral depending on trial and test function.
2. Volume integral depending on test function only.
3. Interior intersection integral depending on trial and test function.
4. Boundary intersection integral depending on trial and test function.
5. Boundary intersection integral depending on test function only.

Also note that no constraints on the function space are necessary in this case.
Dirichlet as well as Neumann boundary conditions are built weakly into the
residual form!

Finally, many types of discontinuous Galerkin finite element methods (DGFEM)
lead to the same five types of integrals and can be applied on general unstructured
conforming as well as nonconforming meshes.

## General Residual Form

The residual form of the cell-centered finite volume method suggests that
all residual forms could be composed of five different types of terms
in the following way:
\begin{equation}
\begin{split}
r(u,v) &=
\sum_{T\in\mathcal{T}_h} \alpha_T^V(R_T u, R_T v)
+ \sum_{T\in\mathcal{T}_h} \lambda_T^V(R_T v) \\
&\qquad+ \sum_{F\in\mathcal{F}_h^i} \alpha_F^S(R_{T_F^-} u,R_{T_F^+} u, R_{T_F^-} v, R_{T_F^+} v)\\
&\qquad+ \sum_{F\in\mathcal{F}_h^{\partial\Omega}} \alpha_F^B(R_{T_F^-} u, R_{T_F^-} v)
+ \sum_{F\in\mathcal{F}_h^{\partial\Omega}} \lambda_F^B(R_{T_F^-} v) .
\end{split}\label{eq:GeneralResidualForm}
\end{equation}
Here, we define the restriction
of a function $u\in U$ to an element by
\begin{equation*}
(R_T u)(x) = u(x) \quad \forall x\in\mathring{T} .
\end{equation*}
Note that the restriction of a function to element $T$ is only defined in
the interior of $T$. On interior intersections $F$, functions may be two-valued
and limits from within the elements $T_F^-, T_F^+$ need to be defined
(when $U$ is the space of element-wise constants that is trivial).

The five terms comprise volume integrals (superscript $V$), interior skeleton integrals
(superscript $S$) and boundary integrals (superscript $B$). Furthermore, the
$\alpha$-terms depend on trial and test functions whereas the $\lambda$-terms only
depend on the test function and involve the data of the PDE.

Each of the five terms $\alpha_T^V$, $\alpha_F^S$, $\alpha_F^B$,
$\lambda_T^V$, $\lambda_F^B$ corresponds to one method on the
local operator.
In addition to the evaluation of residuals also Jacobians and
matrix-free application of Jacobians are needed. This gives rise
to in total $5+3+3=11$ possible methods on a local operator given in the following table:

|    | volume | skeleton | boundary |
|----|----|-----|-----|
|  residual  | `alpha_volume`  <br> `lambda_volume` |`alpha_skeleton` | `alpha_boundary` <br> `lambda_boundary`|
|  Jacobian  | `jacobian_volume`       |`jacobian_skeleton`        | `jacobian_boundary`        |
|  Jac. app. | `jacobian_apply_volume` |`jacobian_apply_skeleton`  | `jacobian_apply_boundary`  |

# Realization in PDELab

## Function `main`

The function `main` is very similar to the one in tutorials 00 and 01 and need not be repeated here.

## Function `driver`

Also the function `driver` is very similar in structure to the one in tutorial 00 and 01. Here we just point out the differences.
The cell-centered finite volume method is based on the space of piecewise constant functions on the mesh $W_h$. The following code segment constructs this function space using the class `P0LocalFiniteElementMap`:

```c++
 // Make grid function space
  typedef Dune::PDELab::P0LocalFiniteElementMap<DF,RF,dim> FEM;
  FEM fem(Dune::GeometryTypes::cube(dim));
  typedef Dune::PDELab::NoConstraints CON;
  typedef Dune::PDELab::ISTL::VectorBackend<> VBE;
  typedef Dune::PDELab::GridFunctionSpace<GV,FEM,CON,VBE> GFS;
  GFS gfs(gv,fem);
  gfs.name("Q0");
```

The constraints class `NoConstraints` is used to express that there are no constraints on the function space.
Now no constraints container type is exported by the grid function space. Instead the class `EmptyTransformation` is used in the grid operator:

```c++
 // Make a global operator
  typedef Dune::PDELab::ISTL::BCRSMatrixBackend<> MBE;
  MBE mbe(2*dim+1); // guess nonzeros per row
  typedef Dune::PDELab::EmptyTransformation CC;
  typedef Dune::PDELab::GridOperator<
    GFS,GFS,  /* ansatz and test space */
    LOP,      /* local operator */
    MBE,      /* matrix backend */
    RF,RF,RF, /* domain, range, jacobian field type*/
    CC,CC     /* constraints for ansatz and test space */
    > GO;
  GO go(gfs,gfs,lop,mbe);
```

Cell-wise data is passed to the `VTKWriter` using its method `addCellData`:

```c++
 // Write VTK output file
  Dune::VTKWriter<GV> vtkwriter(gv,Dune::VTK::conforming);
  typedef Dune::PDELab::VTKGridFunctionAdapter<ZDGF> VTKF;
  vtkwriter.addCellData(std::shared_ptr<VTKF>(new
                                         VTKF(zdgf,"fesol")));
  vtkwriter.write(ptree.get("output.filename","output"),
                  Dune::VTK::appendedraw);
```

These are the only changes to the driver!

## The `Problem` Class

The class `NonlinearPoissonFV` explained below usesvthe same problem class as the class `NonlinearPoissonFEM`. This means that the same problem can be easily solved using the two different methods.

## Local Operator `NonlinearPoissonFV`

The class `NonlinearPoissonFV` implements the element-wise computations of the cell-centered finite volume method. In particular, it provides a full implementation of all possible methods on a local operator including analytic Jacobians. The class has the problem class as a template parameter:

```c++
template<typename Param>
class NonlinearPoissonFV :
  public Dune::PDELab::FullVolumePattern,
  public Dune::PDELab::FullSkeletonPattern,
  public Dune::PDELab::LocalOperatorDefaultFlags
```

The base class `FullSkeletonPattern` provides the local operator with a method coupling all degrees of freedom of two elements sharing an intersection.
In combination with `FullVolumePattern` this provides the sparsity pattern of the matrix.

The only private data member is a reference to an object to the parameter class:

```c++
 Param& param;        // parameter functions
```

The public section begins with a definition of flags controlling assembly of the sparsity pattern

```c++
  // pattern assembly flags
  enum { doPatternVolume = true };
  enum { doPatternSkeleton = true };
```

as well as element contributions:

```c++
 // residual assembly flags
  enum { doLambdaVolume = true };
  enum { doLambdaBoundary = true };
  enum { doAlphaVolume = true };
  enum { doAlphaSkeleton  = true };
  enum { doAlphaBoundary  = true };
```

These five flags specify that all five contributions will be provided. The constructor just gets a reference of the parameter object:

```c++
  NonlinearPoissonFV (Param& param_)
```

### Method `lambda_volume`

continue here 

# Exercises

- [ ] use outlook suggestions
- [ ] somewhere there were exercises for CCFV